##Import Package

In [ ]:
!pip install scikit-criteria==0.2.11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 14.3 MB 14.5 MB/s 
  Created wheel for scikit-criteria: filename=scikit_criteria-0.2.11-py3-none-any.whl size=116439 sha256=52ac60e4cb5055c4851087cef9a1be9015435e9f905087c7bfe1f666ca3d5cf4
  Stored in directory: /root/.cache/pip/wheels/56/d3/68/dda9975c8e89c2cfff66fd38fc05340d1f73f0dc9cc40532e1
Successfully built scikit-criteria


In [ ]:
#import package 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
%matplotlib inline
# !pip install -U scikit-criteria
from skcriteria import Data, MIN, MAX
import statsmodels.api as sm
import statistics

## Desiccated Coconut

In [ ]:
url = "https://github.com/shofi78/RGP-Project/raw/main/dataset/Desiccated%20Coconut%20Competitiveness%20rev1.xlsx"

In [ ]:
df = pd.read_excel(url, "Sheet1")
df["Unit Price"] = df['CIF Value (USD)']/df["Quantity (kg)"]

In [ ]:
df.columns

Index(['Year', 'Importer', 'HS Code', 'Product', 'Quantity (kg)',
       'CIF Value (USD)', 'Xij (USD)', 'Xj(USD)', 'Xiw (USD )', 'Xw (USD)',
       'RCA', 'Avg Tariff (%)', 'Easy Doing Business Rank', 'Avg Distance(km)',
       'Unit Price'],
      dtype='object')

In [ ]:
#Drop kolom yang tidak digunakan
df = df.drop(['Xij (USD)', 'Xj(USD)', 'Xiw (USD )', 'Xw (USD)','Easy Doing Business Rank'], axis=1)

### Proses Wrangling 
- Membuat perankingan dengan Multi Criteria Decision Making menggunakan **skcriteria** package untuk menghasil pilihan terbaik dari beberapa kriteria yang mempengaruhinya

In [ ]:
df.head()

,Year,Importer,HS Code,Product,Quantity (kg),CIF Value (USD),RCA,Avg Tariff (%),Avg Distance(km),Unit Price
0,2021,Australia,80111,"Nuts, edible; coconuts, desiccated",9.407127e+06,2.389052e+07,7.980153,0.0,6954,2.539620
1,2021,China,80111,"Nuts, edible; coconuts, desiccated",1.979375e+07,3.611530e+07,10.381999,0.0,3122,1.824581
2,2021,Dominican Rep.,80111,"Nuts, edible; coconuts, desiccated",2.400073e+07,2.931045e+07,10.476342,20.0,6303,1.221232
3,2021,France,80111,"Nuts, edible; coconuts, desiccated",8.769232e+06,2.625190e+07,131.164060,0.0,7642,2.993638
4,2021,Germany,80111,"Nuts, edible; coconuts, desiccated",2.525530e+07,7.333973e+07,205.618961,0.0,9048,2.903934


In [ ]:

criteria_data = Data(
    df.iloc[:, 6:],                      # the pandas dataframe
    [MAX, MIN, MIN, MAX],                       # direction of goodness for each column
    anames = df['Importer'],        # each entity's name
    cnames = df.columns[6:]             # column name
    )
criteria_data

ALT./CRIT.,RCA (max),Avg Tariff (%) (min),Avg Distance(km) (min),Unit Price (max)
Australia,7.98015,0,6954,2.53962
China,10.382,0,3122,1.82458
Dominican Rep.,10.4763,20,6303,1.22123
France,131.164,0,7642,2.99364
Germany,205.619,0,9048,2.90393
Japan,3.16933,0,4952,2.78581
Malaysia,2.80617,0,1414,1.275
Netherlands,25.3916,0,8892,2.59501
Poland,136,0,6682,2.82432
Rusia,82.7131,0.7,7932,2.45026


In [ ]:
# Melakukan perankingan untuk semua data

from skcriteria.madm import simple
#weighted sum
dm = simple.WeightedSum(mnorm='sum')
dec = dm.decide(criteria_data)
dec

/usr/local/lib/python3.8/dist-packages/skcriteria/norm.py:572: RuntimeWarning: divide by zero encountered in true_divide
  mincrits_inverted = 1.0 / arr[:, mincrits]


ALT./CRIT.,RCA (max),Avg Tariff (%) (min),Avg Distance(km) (min),Unit Price (max),Rank
Australia,7.98015,0,6954,2.53962,1
China,10.382,0,3122,1.82458,2
Dominican Rep.,10.4763,20,6303,1.22123,17
France,131.164,0,7642,2.99364,3
Germany,205.619,0,9048,2.90393,4
Japan,3.16933,0,4952,2.78581,5
Malaysia,2.80617,0,1414,1.275,6
Netherlands,25.3916,0,8892,2.59501,7
Poland,136,0,6682,2.82432,8
Rusia,82.7131,0.7,7932,2.45026,14


In [ ]:
# Mengubah tipe data dari array to series terlebih dahulu
rank = pd.Series(dec.rank_, name='rank')

# Menggabungkan data 
df_rank= pd.concat([df,rank],axis=1)

# Mengurutkan ranking dari tertinggi ke terendah
df_rank.sort_values(by=['rank'], ignore_index=True, inplace=True)

In [ ]:
df_rank

,Year,Importer,HS Code,Product,Quantity (kg),CIF Value (USD),RCA,Avg Tariff (%),Avg Distance(km),Unit Price,rank
0,2021,Australia,80111,"Nuts, edible; coconuts, desiccated",9.407127e+06,2.389052e+07,7.980153,0.0,6954,2.539620,1
1,2021,China,80111,"Nuts, edible; coconuts, desiccated",1.979375e+07,3.611530e+07,10.381999,0.0,3122,1.824581,2
2,2021,France,80111,"Nuts, edible; coconuts, desiccated",8.769232e+06,2.625190e+07,131.164060,0.0,7642,2.993638,3
3,2021,Germany,80111,"Nuts, edible; coconuts, desiccated",2.525530e+07,7.333973e+07,205.618961,0.0,9048,2.903934,4
4,2021,Japan,80111,"Nuts, edible; coconuts, desiccated",4.800845e+06,1.337424e+07,3.169335,0.0,4952,2.785810,5
5,2021,Malaysia,80111,"Nuts, edible; coconuts, desiccated",1.327796e+07,1.692934e+07,2.806171,0.0,1414,1.274996,6
6,2021,Netherlands,80111,"Nuts, edible; coconuts, desiccated",3.440113e+07,8.927129e+07,25.391594,0.0,8892,2.595010,7
7,2021,Poland,80111,"Nuts, edible; coconuts, desiccated",1.021857e+07,2.886046e+07,136.000010,0.0,6682,2.824316,8
8,2021,Singapore,80111,"Nuts, edible; coconuts, desiccated",3.325060e+07,2.604357e+07,27.167782,0.0,770,0.783251,9
9,2021,Spain,80111,"Nuts, edible; coconuts, desiccated",6.832213e+06,1.793195e+07,37.593825,0.0,8060,2.624618,10


In [ ]:
# save to excel and visualization
from google.colab import files
df_rank.to_excel('dc_rangking.xlsx')
files.download('dc_rangking.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Briquette

In [ ]:
url = "https://github.com/shofi78/RGP-Project/raw/main/dataset/Briquette%20Competitiveness.xlsx"

In [ ]:
df2 = pd.read_excel(url)
df2["Unit Price"] = df2["Value (USD)"]/df2["Quantity (kg)"]
df2.head()

,Year,Importer,HS Code,Product,Quantity (kg),Value (USD),Xij (USD),Xj(USD x 103 ),Xiw (USD ),Xw (USD x 103 ),RCA,Avg Tariff (%),Distance (km),Unit Price
0,2021,USA,440290,Wood; charcoal of wood other than bamboo (incl...,1.830100e+08,119512160,6250000,25665933,119512160,2937064484,5.984451,0,14972,0.653036
1,2021,Japan,440290,Wood; charcoal of wood other than bamboo (incl...,1.218080e+08,105104155,14504000,17736773,105104155,772678811,6.011631,0,4808,0.862867
2,2021,Germany,440290,Wood; charcoal of wood other than bamboo (incl...,1.491845e+08,97653584,5863000,2799691,97653584,1421195357,30.477223,0,11014,0.654583
3,2021,China,440290,Wood; charcoal of wood other than bamboo (incl...,2.614294e+08,87063526,7448000,53713561,87063526,2675680064,4.261413,0,4181,0.333029
4,2021,Rep. of Korea,440290,Wood; charcoal of wood other than bamboo (incl...,9.884540e+07,85459789,19797000,8408928,85459789,615034495,16.943238,0,4309,0.864580


In [ ]:
df = pd.read_excel(url, "Sheet1")
df["Unit Price"] = df['CIF Value (USD)']/df["Quantity (kg)"]

In [ ]:

criteria_data = Data(
    df2.iloc[:, 10:],                      # the pandas dataframe
    [MAX, MIN, MIN, MAX],                       # direction of goodness for each column
    anames = df2['Importer'],        # each entity's name
    cnames = df2.columns[10:]             # column name
    )
criteria_data

ALT./CRIT.,RCA (max),Avg Tariff (%) (min),Distance (km) (min),Unit Price (max)
USA,5.98445,0,14972,0.653036
Japan,6.01163,0,4808,0.862867
Germany,30.4772,0,11014,0.654583
China,4.26141,0,4181,0.333029
Rep. of Korea,16.9432,0,4309,0.86458
Saudi Arabia,57.0333,0,7907,0.603892
United Kingdom,13.9533,0,11770,0.668033
France,3.2607,0,11728,0.797971
Poland,0.720981,0,10405,0.380291
Netherlands,47.8092,0,11340,0.714347


In [ ]:
# Mengubah tipe data dari array to series terlebih dahulu
rank = pd.Series(dec.rank_, name='rank')

# Menggabungkan data 
df2_rank= pd.concat([df2,rank],axis=1)

# Mengurutkan ranking dari tertinggi ke terendah
df2_rank.sort_values(by=['rank'], ignore_index=True, inplace=True)

In [ ]:
df2_rank

,Year,Importer,HS Code,Product,Quantity (kg),Value (USD),Xij (USD),Xj(USD x 103 ),Xiw (USD ),Xw (USD x 103 ),RCA,Avg Tariff (%),Distance (km),Unit Price,rank
0,2021,USA,440290,Wood; charcoal of wood other than bamboo (incl...,1.830100e+08,119512160,6250000,25665933,119512160,2937064484,5.984451,0,14972,0.653036,1
1,2021,Japan,440290,Wood; charcoal of wood other than bamboo (incl...,1.218080e+08,105104155,14504000,17736773,105104155,772678811,6.011631,0,4808,0.862867,2
2,2021,China,440290,Wood; charcoal of wood other than bamboo (incl...,2.614294e+08,87063526,7448000,53713561,87063526,2675680064,4.261413,0,4181,0.333029,3
3,2021,Rep. of Korea,440290,Wood; charcoal of wood other than bamboo (incl...,9.884540e+07,85459789,19797000,8408928,85459789,615034495,16.943238,0,4309,0.864580,4
4,2021,Saudi Arabia,440290,Wood; charcoal of wood other than bamboo (incl...,1.376932e+08,83151832,47875000,1537819,83151832,152333959,57.033296,0,7907,0.603892,5
5,2021,United Kingdom,440290,Wood; charcoal of wood other than bamboo (incl...,9.661612e+07,64542776,1927000,1472630,64542776,688236539,13.953339,0,11770,0.668033,6
6,2021,France,440290,Wood; charcoal of wood other than bamboo (incl...,8.052244e+07,64254581,306000,1025147,64254581,701907963,3.260704,0,11728,0.797971,7
7,2021,Poland,440290,Wood; charcoal of wood other than bamboo (incl...,1.255036e+08,47727838,63000,614150,47727838,335451322,0.720981,0,10405,0.380291,8
8,2021,Norway,440290,Wood; charcoal of wood other than bamboo (incl...,3.891976e+07,30065439,0,53345,30065439,98636184,0.000000,0,10924,0.772498,9
9,2021,Greece,440290,Wood; charcoal of wood other than bamboo (incl...,5.276676e+07,29270397,2395000,226818,29270397,75983832,27.410729,0,10248,0.554713,10


In [ ]:
# save to excel and visualization
from google.colab import files
df2_rank.to_excel('briquette_rangking.xlsx')
files.download('briquette_rangking.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>